In [83]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
import plotly.express as px


#ek.set_app_key("320e7257c23945729695a106cedb2b9cb72d3207") # Elena
ek.set_app_key("69de0ce7c5e2468e90a6058ce79f6558ebe3ee30") # Alexander
#ek.set_app_key("f47c330480d74c598b7e8ebc2539424e91764dd8") # Ginevra


https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

# Economic Story

**TR.RevenueActValue.date** - prints the datetime

## Part 1: Accuracy

1. **TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

2. **TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> this is a analyst forecast variable

3. **TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> forecast error between actual EPS and TR.EPSMean

<img src="./Pictures/Screenshot 2023-02-15 185222.png" width=500 height=300 />

## Part 2: Factors

### Firm charateristics

1. **TR.RevenueActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Revenue (or Sales) is a corporation's net revenue, generally derived from core business activities. For non-financial companies, the calculation of net revenue (or net turnover) in most markets generally involves subtracting transportation and related operational costs from gross revenue/sales. Revenue recognition practices vary significantly from market to market, though generally the recording of revenue is based upon sales invoices issued (or anticipated for forecast purposes) during the accounting period.

2. **TR.F.EV** - Enterprise Value is the Company's total economic value OR the theoretical takeover price, often used as an alternative to market capitalization. Enterprise Value represents Market Capitalization TR.F.MktCap + Debt including Preferred Equity & Minority Interest - Total [TR.F.DebtlnclPrefEqMinlntrTot] (-) Cash & Short Term Investments - Total [TR.F.CashSTlnvstTot]. Market Capitalization [TR.F.MktCap] is required. The data item is calculated for all periodicities. It is applicable to all industries.

3. **TR.LTGMean** - Long Term Growth - Mean:  The statistical average of all broker estimates determined to be on the majority accounting basis. Long-term growth is an estimate of the compound average rate of EPS growth an analyst expects over a period of three to five ears.

##### Loss Firm Status (0 or 1)

4. **TR.F.NetIncAfterTax** Net Income after Tax [SIAT] represents the income/expense after all operating and non-operating income and expense, reserves, income taxes, but before equity in earnings, minority interest, extraordinary items, after-tax adjustments, discontinued operations and preferred dividends. Applicable to all Industries. Net Income after Tax [SIAT] includes: • Net Income after Tax [XIAT]

### Analysts
1. **TR.NumberOfAnalysts** - Number of sell-side analysts covering the security.

2. **TR.RecMean** - Recommendation Numeric Mean based on the Standard Scale Of Strong Buy (1 ), Buy (2), Hold (3), Sell (4), and Strong Sell (5).

3. change in recommendation

### Macro characteristics

1. **TR.WACCInflationAdjRiskFreeRate** -  Inflation adjusted risk free rate used is Of the primary index for the domicile country Of the company. Inflation adjusted risk free rate (Rf) is calculated from the US 10-year treasury yield plus the difference between in the 10-year forecasted inflation rate between the given country and the US.



### Return Potential

1. **TR.PriceTargetMean** The statistical average of all broker estimates determined to be on the majority accounting basis. Price target is the projected price level forecasted by the analyst within a specific time horizon.

1. **TR.PriceClose** The latest available closing price. If there are no trades for the most recent completed tradable day, the most recent prior tradable day with trading activity is used, provided the last tradable day for the instrument is within 378 completed calendar days.

### More difficult Factors (yearly):

#### ESG

1. **TR.TRESGScore** Refinitiv ESG Score is an overall company score based on the self-reported information in the environmental, social and corporate governance pillars.

#### Proportion of independent Directors

1. **TR.AnalyticIndepBoard** - Independent Board Members - Percentage of independent board members as reported by the company.

2. **TR.GovernancePillarScore** - Governance Pillar Score - The corporate governance pillar measures a company's systems and processes, which ensure that its board members and executives act in the best interests of its long term shareholders. It reflects a company's capacity, through its use of best management practices, to direct and control its rights and responsibilities through the creation of incentives, as well as checks and balances in order to generate long term shareholder value.

## Part 3: Regression

### continue..

In [2]:
economic_story = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise", 'TR.RevenueActValue', "TR.F.EV", "TR.LTGMean",  "TR.NumberOfAnalysts", "TR.RecMean", "TR.RecMean(SDate=-2,EDate=-41)", "TR.WACCInflationAdjRiskFreeRate", "TR.F.NetIncAfterTax", "TR.PriceTargetMean", "TR.PriceClose"]

In [50]:
df, e = ek.get_data('0#.SPX', economic_story, parameters = {'SDate':'-1','EDate':'-40','Period':'FQ0','Frq':'FQ'})
df["Date"] = pd.to_datetime(df["Date"])
df

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1134: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1153: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000.0,14061600372.299999,<NA>,10,2.27273,2.2,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000.0,15561870705.290001,<NA>,9,2.2,2.0,3.092855,307205000.0,435.375,351.23
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000.0,18436823034.200001,<NA>,9,2.0,2.3,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000.0,23848820586.0,<NA>,8,2.3,2.3,1.515266,107542000.0,571.0,566.0
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000.0,17699543822.66,<NA>,8,2.3,2.3,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-31 08:30:00+00:00,0.69,0.68,1.471,1583900000.0,5553067502.24,7.0,8,2.33333,2.625,<NA>,42100000.0,54.625,50.19
20116,AVY.N,2013-10-25 08:30:00+00:00,0.69,0.63833,8.095,1504900000.0,5029514353.46,7.0,7,2.625,2.875,<NA>,62000000.0,47.71429,43.52
20117,AVY.N,2013-07-23 08:30:00+00:00,0.71,0.7025,1.068,1552300000.0,5436229328.84,14.35,8,2.875,2.875,<NA>,70800000.0,45.0,42.76
20118,AVY.N,2013-04-24 08:30:00+00:00,0.59,0.57571,2.482,1498900000.0,5459128200.46,14.35,7,2.875,3.0,<NA>,66800000.0,42.57143,43.07


In [51]:
#df = df.dropna()
#df

In [52]:
cols = []
count = 1
for column in df.columns:
    if column == 'Recommendation - Mean (1-5)':
        cols.append(f'Recommendation - Mean (1-5)_{count}')
        count+=1
        continue
    cols.append(column)
df.columns = cols
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5)_1,Recommendation - Mean (1-5)_2,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000.0,14061600372.299999,<NA>,10,2.27273,2.2,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000.0,15561870705.290001,<NA>,9,2.2,2.0,3.092855,307205000.0,435.375,351.23
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000.0,18436823034.200001,<NA>,9,2.0,2.3,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000.0,23848820586.0,<NA>,8,2.3,2.3,1.515266,107542000.0,571.0,566.0
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000.0,17699543822.66,<NA>,8,2.3,2.3,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-31 08:30:00+00:00,0.69,0.68,1.471,1583900000.0,5553067502.24,7.0,8,2.33333,2.625,<NA>,42100000.0,54.625,50.19
20116,AVY.N,2013-10-25 08:30:00+00:00,0.69,0.63833,8.095,1504900000.0,5029514353.46,7.0,7,2.625,2.875,<NA>,62000000.0,47.71429,43.52
20117,AVY.N,2013-07-23 08:30:00+00:00,0.71,0.7025,1.068,1552300000.0,5436229328.84,14.35,8,2.875,2.875,<NA>,70800000.0,45.0,42.76
20118,AVY.N,2013-04-24 08:30:00+00:00,0.59,0.57571,2.482,1498900000.0,5459128200.46,14.35,7,2.875,3.0,<NA>,66800000.0,42.57143,43.07


In [53]:
df["Recommendation change"] = df["Recommendation - Mean (1-5)_1"] - df["Recommendation - Mean (1-5)_2"]
df.insert(10, "Recommendation change", df.pop("Recommendation change"))
df = df.rename(columns={"Recommendation - Mean (1-5)_1": "Recommendation - Mean (1-5)"})
df = df.drop("Recommendation - Mean (1-5)_2", axis=1)
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000.0,14061600372.299999,<NA>,10,2.27273,0.07273,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000.0,15561870705.290001,<NA>,9,2.2,0.2,3.092855,307205000.0,435.375,351.23
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000.0,18436823034.200001,<NA>,9,2.0,-0.3,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000.0,23848820586.0,<NA>,8,2.3,0.0,1.515266,107542000.0,571.0,566.0
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000.0,17699543822.66,<NA>,8,2.3,0.0,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-31 08:30:00+00:00,0.69,0.68,1.471,1583900000.0,5553067502.24,7.0,8,2.33333,-0.29167,<NA>,42100000.0,54.625,50.19
20116,AVY.N,2013-10-25 08:30:00+00:00,0.69,0.63833,8.095,1504900000.0,5029514353.46,7.0,7,2.625,-0.25,<NA>,62000000.0,47.71429,43.52
20117,AVY.N,2013-07-23 08:30:00+00:00,0.71,0.7025,1.068,1552300000.0,5436229328.84,14.35,8,2.875,0.0,<NA>,70800000.0,45.0,42.76
20118,AVY.N,2013-04-24 08:30:00+00:00,0.59,0.57571,2.482,1498900000.0,5459128200.46,14.35,7,2.875,-0.125,<NA>,66800000.0,42.57143,43.07


In [54]:
df['New Date'] = pd.Series()

for i in range(len(df)):
    if df.loc[i, "Date"].month == 1:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 2:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 3:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 4:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 5:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 6:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 7:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 8:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 9:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 10:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)
    elif df.loc[i, "Date"].month == 11:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)
    elif df.loc[i, "Date"].month == 12:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)

df.insert(2, "New Date", df.pop("New Date"))
df["New Date"] = pd.to_datetime(df["New Date"]).dt.date

df

C:\Users\alexa\AppData\Local\Temp\ipykernel_27636\2145643216.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['New Date'] = pd.Series()


,Instrument,Date,New Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-20 07:00:00+00:00,2022-10-01,4.78,4.5875,4.196,1615339000.0,14061600372.299999,<NA>,10,2.27273,0.07273,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-21 07:00:00+00:00,2022-07-01,7.63,7.517,1.503,2055818000.0,15561870705.290001,<NA>,9,2.2,0.2,3.092855,307205000.0,435.375,351.23
2,POOL.OQ,2022-04-21 07:00:00+00:00,2022-04-01,4.23,3.14867,34.342,1412650000.0,18436823034.200001,<NA>,9,2.0,-0.3,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-02-17 07:00:00+00:00,2022-01-01,2.63,1.875,40.267,1035557000.0,23848820586.0,<NA>,8,2.3,0.0,1.515266,107542000.0,571.0,566.0
4,POOL.OQ,2021-10-21 07:00:00+00:00,2021-10-01,4.51,3.84833,17.194,1411448000.0,17699543822.66,<NA>,8,2.3,0.0,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-31 08:30:00+00:00,2014-01-01,0.69,0.68,1.471,1583900000.0,5553067502.24,7.0,8,2.33333,-0.29167,<NA>,42100000.0,54.625,50.19
20116,AVY.N,2013-10-25 08:30:00+00:00,2013-10-01,0.69,0.63833,8.095,1504900000.0,5029514353.46,7.0,7,2.625,-0.25,<NA>,62000000.0,47.71429,43.52
20117,AVY.N,2013-07-23 08:30:00+00:00,2013-07-01,0.71,0.7025,1.068,1552300000.0,5436229328.84,14.35,8,2.875,0.0,<NA>,70800000.0,45.0,42.76
20118,AVY.N,2013-04-24 08:30:00+00:00,2013-04-01,0.59,0.57571,2.482,1498900000.0,5459128200.46,14.35,7,2.875,-0.125,<NA>,66800000.0,42.57143,43.07


In [70]:
characteristics = df.loc[:, ["Instrument", "New Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]] #  "Enterprise Value",
analysts = df.loc[:, ["Instrument", "New Date", "Earnings Per Share - Actual Surprise", "Number of Analysts", "Recommendation - Mean (1-5)", 'Recommendation change']]
macro = df.loc[:, ["Instrument", "New Date", "Earnings Per Share - Actual Surprise", "WACC Inflation Adjusted Risk Free Rate, (%)"]]

In [71]:
characteristics["Net Income after Tax"] = characteristics["Net Income after Tax"].fillna(1)
characteristics["loss firm"] = [1 if x > 1 else -1 if x <= 0 else 0 for x in characteristics["Net Income after Tax"]]
characteristics = characteristics.drop("Net Income after Tax", axis=1)
characteristics

,Instrument,New Date,Earnings Per Share - Actual Surprise,Revenue - Actual,loss firm
0,POOL.OQ,2022-10-01,4.196,1615339000.0,1
1,POOL.OQ,2022-07-01,1.503,2055818000.0,1
2,POOL.OQ,2022-04-01,34.342,1412650000.0,1
3,POOL.OQ,2022-01-01,40.267,1035557000.0,1
4,POOL.OQ,2021-10-01,17.194,1411448000.0,1
...,...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,1583900000.0,1
20116,AVY.N,2013-10-01,8.095,1504900000.0,1
20117,AVY.N,2013-07-01,1.068,1552300000.0,1
20118,AVY.N,2013-04-01,2.482,1498900000.0,1


In [49]:
characteristics = characteristics.dropna()
characteristics["loss firm"] = [-1 if x > 0 else 1 if x < -1 else 0 for x in characteristics["Net Income after Tax"]]
characteristics

,Instrument,New Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Net Income after Tax,loss firm
0,POOL.OQ,2022-10-01,4.196,1615339000.0,14061600372.299999,189965000.0,-1
1,POOL.OQ,2022-07-01,1.503,2055818000.0,15561870705.290001,307205000.0,-1
2,POOL.OQ,2022-04-01,34.342,1412650000.0,18436823034.200001,179203000.0,-1
3,POOL.OQ,2022-01-01,40.267,1035557000.0,23848820586.0,107542000.0,-1
4,POOL.OQ,2021-10-01,17.194,1411448000.0,17699543822.66,184573000.0,-1
...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,1583900000.0,5553067502.24,42100000.0,-1
20116,AVY.N,2013-10-01,8.095,1504900000.0,5029514353.46,62000000.0,-1
20117,AVY.N,2013-07-01,1.068,1552300000.0,5436229328.84,70800000.0,-1
20118,AVY.N,2013-04-01,2.482,1498900000.0,5459128200.46,66800000.0,-1


In [10]:
analysts

,Instrument,New Date,Earnings Per Share - Actual Surprise,Number of Analysts,Recommendation - Mean (1-5),Recommendation change
0,POOL.OQ,2022-10-01,4.196,10,2.27273,0.07273
1,POOL.OQ,2022-07-01,1.503,9,2.2,0.2
2,POOL.OQ,2022-04-01,34.342,9,2.0,-0.3
3,POOL.OQ,2022-01-01,40.267,8,2.3,0.0
4,POOL.OQ,2021-10-01,17.194,8,2.3,0.0
...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,8,2.33333,-0.29167
20116,AVY.N,2013-10-01,8.095,7,2.625,-0.25
20117,AVY.N,2013-07-01,1.068,8,2.875,0.0
20118,AVY.N,2013-04-01,2.482,7,2.875,-0.125


In [11]:
macro

,Instrument,New Date,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)"
0,POOL.OQ,2022-10-01,4.196,3.747421
1,POOL.OQ,2022-07-01,1.503,3.092855
2,POOL.OQ,2022-04-01,34.342,2.325202
3,POOL.OQ,2022-01-01,40.267,1.515266
4,POOL.OQ,2021-10-01,17.194,1.527139
...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,<NA>
20116,AVY.N,2013-10-01,8.095,<NA>
20117,AVY.N,2013-07-01,1.068,<NA>
20118,AVY.N,2013-04-01,2.482,<NA>


In [72]:
yearly = ["TR.TRESGScore.date", "TR.TRESGScore", "TR.GovernancePillarScore", "TR.AnalyticIndepBoard"]

In [88]:
economic_story_yearly, e = ek.get_data('0#.SPX', yearly, parameters = {'SDate':'0','EDate':'-20','Period':'FY0','Frq':'FY'})

In [89]:
economic_story_yearly["Date"] = pd.to_datetime(economic_story_yearly["Date"]).dt.date
economic_story_yearly = economic_story_yearly.dropna()
economic_story_yearly

,Instrument,Date,ESG Score,Governance Pillar Score,Independent Board Members
0,POOL.OQ,2021-12-31,48.718747,79.856229,80.0
1,POOL.OQ,2020-12-31,40.485976,69.05684,88.888889
2,POOL.OQ,2019-12-31,44.618552,80.983486,88.888889
3,POOL.OQ,2018-12-31,42.83681,66.013223,87.5
4,POOL.OQ,2017-12-31,39.816484,54.70234,75.0
...,...,...,...,...,...
10513,AVY.N,2007-12-29,32.044479,67.581987,81.818182
10515,AVY.N,2005-12-31,4.927093,3.622322,80.0
10516,AVY.N,2005-01-01,10.087481,11.414989,66.666667
10517,AVY.N,2003-12-27,8.450697,5.399386,54.545455


In [ ]:
# df['ESG Score'] = pd.Series()
# df['Governance Pillar Score'] = pd.Series()
# df['Independent Board Members'] = pd.Series()
#
# for i in range(len(df2)):
#     for j in range(len(df)):
#         if df.loc[j, "Instrument"] == df2.loc[i, "Instrument"] and df.loc[j, "New Date"].year == df2.loc[i, "Date"].year:
#             df.loc[j, "ESG Score"] = df2.loc[i, "ESG Score"]
#             df.loc[j, "Governance Pillar Score"] = df2.loc[i, "Governance Pillar Score"]
#             df.loc[j, "Independent Board Members"] = df2.loc[i, "Independent Board Members"]
#
# df